In [ ]:
%pip install pygame

In [ ]:
from pong import *

In [ ]:
import os
import pandas as pd
import tensorflow as tf


def generate_dataset(save_to_disk=False):
    '''Generates a dataset of frames and states from Pong.'''
    pong = Pong(sound_enabled=False)
    df = pd.DataFrame(columns=['left_x', 'left_y', 'right_x', 'right_y', 'ball_x', 'ball_y', 'ball_vx', 'ball_vy', 'left_score', 'right_score'])

    screenshots = []
    i = 0

    # Make sure the image directory exists.
    if save_to_disk:
        os.makedirs(DATA_DIRECTORY_NAME, exist_ok=True)

    for p1_score, p2_score in SCORE_COMBINATIONS:
        for state in range(STATES_PER_SCORE):
            pong.restart()  # Start a new phase of the game.

            # Generate a random state.
            pong.p1.y = random.uniform(PADDLE_HEIGHT, SCREEN_HEIGHT)
            pong.p1.score = p1_score

            pong.p2.y = random.uniform(PADDLE_HEIGHT, SCREEN_HEIGHT)
            pong.p2.score = p2_score

            pong.ball.x = random.uniform(GOAL_PADDING, SCREEN_WIDTH - GOAL_PADDING)
            pong.ball.y = random.uniform(BALL_SIZE, SCREEN_HEIGHT)

            overrides = {}
            pong.paused = False

            # Simulate the game for a couple timesteps.
            for _ in range(TIMESTEPS):
                pong.update(IDEAL_DT, overrides)
                state, screenshot = pong.capture()

                if save_to_disk:
                    pygame.image.save(screenshot, f'{DATA_DIRECTORY_NAME}/{i}.png')

                # Create a new record.
                df.loc[len(df)] = state
                screenshot = pygame.surfarray.array3d(screenshot).astype(np.uint8)
                screenshot = np.transpose(screenshot, (1, 0, 2))  # (height, width, 3)
                screenshot = screenshot[..., 0] / 255.0 # or 1 or 2, since R = G = B
                screenshot = screenshot.reshape(-1)
                screenshots.append(screenshot)

                # Simulate player input that simply chases the ball.
                overrides = {}

                for paddle in pong.paddles:                
                    if pong.ball.y >= paddle.y :  # Ball is above the paddle, move up.
                        overrides[paddle.up] = True
                    elif pong.ball.y - BALL_SIZE <= paddle.y - PADDLE_HEIGHT:  # Ball is below the paddle, move down
                        overrides[paddle.down] = True

                i += 1

    if save_to_disk:
        df.to_csv(SPREADSHEET_FILEPATH, index=False)

    # Convert to numpy arrays.
    states = df.to_numpy()
    screenshots = np.stack(screenshots)

    return states, screenshots

In [ ]:
X, y = generate_dataset(True)

In [ ]:
def load_dataset():
    '''Loads in a previously generated dataset from disk.'''
    # Read in the state data.
    df = pd.read_csv(SPREADSHEET_FILEPATH)

    # Load in all the screenshots.
    screenshots = []

    for i, _ in df.iterrows():
        path = f'{DATA_DIRECTORY_NAME}/{i}.png'
        screenshot = tf.io.read_file(path)
        screenshot = tf.image.decode_png(screenshot, channels=1).numpy() / 255
        screenshot = tf.squeeze(screenshot, axis=-1)
        screenshot = tf.reshape(screenshot, [-1])
        screenshots.append(screenshot)

    # Convert to numpy arrays.
    states = df.to_numpy()
    screenshots = np.stack(screenshots)

    return states, screenshots

In [ ]:
from itertools import product


def generate_configs(options):
    '''Generates the Cartesian product of the supplied dictionary.'''
    keys = options.keys()
    values = options.values()
    return [dict(zip(keys, combo)) for combo in product(*values)]


def stringify(config):
    '''Converts a model config to a single string based its values.'''
    return '_'.join([str(option).replace(' ','') for option in config.values()])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv2D, MaxPooling2D, Flatten


PIXELS = SCREEN_WIDTH * SCREEN_HEIGHT


def generate_fcnn_models(options):
    '''Generates a list of compiled feed-forward neural networks.'''
    models = []

    for config in generate_configs(options):
        layers = []

        # Add the hidden layers.
        for neurons in config['neurons']:
            layers.append(Dense(neurons, activation=config['activation']))

        # Add the output layer.
        layers.append(Dense(PIXELS, activation='sigmoid'))

        model = Sequential(layers)
        model.compile(optimizer=config['optimizer'], loss='binary_crossentropy')
        name = 'dense_' + stringify(config)

        models.append((model, name))

    return models[:3]


def generate_rnn_models(options):
    '''Generates a list of compiled recurrent neural networks.'''
    models = []

    for config in generate_configs(options):
        layers = []

        # LSTM layers.
        for i, neurons in enumerate(config['bottom_layers'], 1):
            return_sequences = i != len(config['bottom_layers'])
            layers.append(LSTM(neurons, activation='tanh', dropout=config['dropout'], return_sequences=return_sequences))

        # Decoder layers.
        for neurons in config['top_layers']:
            layers.append(Dense(neurons, activation='relu'))

        # Output layer.
        layers.append(Dense(PIXELS, activation='sigmoid'))

        model = Sequential(layers)
        model.compile(optimizer=config['optimizer'], loss='binary_crossentropy')
        name = 'rnn__' + stringify(config)

        models.append((model, name))

    return models[:3]


def generate_cnn_models(options):
    '''Generates a list of compiled convolutional neural networks.'''
    models = []

    for config in generate_configs(options):
        layers = []

        # Convolutional and pooling layers.
        for neurons in config['bottom_layers']:
            layers.append(Conv2D(neurons, config['kernel_size'], activation='relu', padding='same'))
            layers.append(MaxPooling2D(config['pool_size'], padding='same'))

        # Output must be flattened.
        layers.append(Flatten())

        # Decoder.
        for neurons in config['top_layers']:
            layers.append(Dense(neurons, activation='relu'))

        # Output layer.
        layers.append(Dense(PIXELS, activation='sigmoid'))

        model = Sequential(layers)
        model.compile(optimizer='adamw', loss='binary_crossentropy')
        name = 'cnn__' + stringify(config)

        models.append((model, name))

    return models[:3]

In [ ]:
import time

from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tqdm.notebook import tqdm


@dataclass
class Result:
    '''A small struct-like class for recording a model's performance.'''
    name: str
    score: float
    latency: float


def train(models, X_train, y_train, X_test, y_test):
    '''Trains a set of models on the supplied data.'''
    results = []

    for model, name in tqdm(models, desc='Training Models'):
        model_filepath = f'{TRAINING_DIRECTORY_NAME}/{name}.keras'

        # Callbacks to help increase model performance.
        checkpointer = ModelCheckpoint(filepath=model_filepath, verbose=0, save_best_only=True)
        stopper = EarlyStopping(patience=10, verbose=1)

        # Train the model.
        model.fit(X_train, y_train, epochs=10, verbose=0, validation_data=(X_test, y_test), callbacks=[checkpointer, stopper])

        # Swap to the best version for evaluation.
        model.load_weights(model_filepath)

        # Evaluate the model's performance.
        predictions = model.predict(X_test, verbose=0)
        predictions = np.round(predictions)
        score = metrics.f1_score(predictions, y_test, average="weighted", zero_division=0)

        # Measure the model's latency.
        start = time.time()

        for i in range(10):
            _ = model.predict(X_train[i:i+1], verbose=0)

        latency = (time.time() - start) / 10

        results.append(Result(name, score, latency))

    return results

In [ ]:
import shutil


column_titles = f'| {"Model Rank":^12} | {"Model Name":^36} | {"F1-Score":^14} | {"Latency":^11} |'
separator = '=' * len(column_titles)


def select_best_and_worst_results(results, metric, best_filepath):
    '''Returns the best and worst models/results based on the given metric (score or latency).'''
    results.sort(key=metric)
    best_result = results[0]
    worst_result = results[-1]

    # Save the best model.
    shutil.copy2(f'{TRAINING_DIRECTORY_NAME}/{best_result.name}.keras', best_filepath)

    return best_result, worst_result


def print_header(title):
    '''Prints a formatted header for displaying results.'''
    title = f'|{title.center(len(column_titles) - 2)}|'

    for section in [separator, title, separator, column_titles, separator]:
        print(section)


def print_top_results(title, results, count):
    '''Prints the results of the top models.'''
    print_header(title)

    for i, result in enumerate(results[:count]):
        # Names that are too long must be truncated with '...'.
        name = result.name[:33] + '...' if len(result.name) > 22 else result.name
        print(f'| {i + 1:^12} | {name:^36} | {result.score:^14.5f} | {result.latency:^11.5f} |')


def evaluate(results, best_model_filepaths, count=5):
    '''Shows the best scoring and fastest latency results in a table.'''
    os.makedirs(BEST_MODEL_DIRECTORY_PATH, exist_ok=True)

    # Print best results by score.
    score_metric = lambda result: -result.score
    best_score_result, worst_score_result = select_best_and_worst_results(results, score_metric, best_model_filepaths[0])
    score_range = best_score_result.score - worst_score_result.score
    print_top_results('Best F1-Score', results, count)
    print(f'| {"-":^12} | {"F1-SCORE RANGE":^36} | {score_range:^14.5f} | {"-":^11} |')

    # Print best results by latency.
    latency_metric = lambda result: result.latency
    best_latency_result, worst_latency_result = select_best_and_worst_results(results, latency_metric, best_model_filepaths[1])
    latency_range = worst_latency_result.latency - best_latency_result.latency

    print_top_results('Best Latency ', results, count)
    print(f'| {"-":^12} | {"LATENCY RANGE":^36} | {"-":^14} | {latency_range:^11.5f} |')

    print(separator)

In [ ]:
from sklearn.model_selection import train_test_split


X, y = generate_dataset()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
from sklearn.model_selection import train_test_split


dense_options = {
    'neurons': [[], [128], [128, 256], [128, 256, 512], [128, 256, 512, 1024], [256, 512, 1024, 2048]],
    'activation': [None, 'relu', 'tanh', 'sigmoid'],
    'optimizer': ['adam', 'adamw', 'sgd']
}

dense_models = generate_fcnn_models(dense_options)
results = train(dense_models, X_train, y_train, X_test, y_test)

In [ ]:
evaluate(results, FCNN_BEST_FILEPATHS)

In [ ]:
X = X.reshape(SAMPLES, TIMESTEPS, FEATURES)
y = y.reshape(SAMPLES, TIMESTEPS, PIXELS)[:, -1, :]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
rnn_options = {
    'bottom_layers': [[64], [64, 256], [128, 256, 512]],
    'top_layers': [[], [128, 512], [1024], [4096]],
    'dropout': [0.0, 0.1],
    'optimizer': ['adam', 'adamw', 'sgd']
}

rnn_models = generate_rnn_models(rnn_options)
results = train(rnn_models, X_train, y_train, X_test, y_test)

In [ ]:
evaluate(results, RNN_BEST_FILEPATHS)

In [ ]:
X = X.reshape(SAMPLES, TIMESTEPS, FEATURES, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
cnn_options = {
    'bottom_layers': [[32], [64], [32, 64], [32, 64, 128]],
    'top_layers': [[], [128, 512], [128, 512, 1024], [8096]],
    'pool_size': [(2, 2), (3, 3), (5, 5)],
    'kernel_size': [(3, 3), (5, 5)],
}

cnn_models = generate_cnn_models(cnn_options)
results = train(cnn_models, X_train, y_train, X_test, y_test)

In [ ]:
evaluate(results, CNN_BEST_FILEPATHS)

In [ ]:
import traceback


try:
    main()
except Exception as e:
    traceback.print_exc()